In [ ]:
import FinanceDataReader as fdr
import pandas as pd
import numpy as np
import datetime

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor


In [ ]:
def prep_data(stock_code, predict_date):
    predict_date = pd.to_datetime(predict_date)
    end_date = predict_date - datetime.timedelta(days=1) # 예측날짜의 하루전의 테이터까지만 사용
    start_date = end_date - datetime.timedelta(days=period) # 1년 전의 데이터까지 사용
    
    start_weekday = pd.to_datetime(start_date).weekday()
    max_weeknum = pd.to_datetime(end_date).strftime('%V')
    Business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])

    stock = fdr.DataReader(stock_code, start=start_date, end=end_date).reset_index()
    stock_data = pd.merge(Business_days, stock, how='outer')
    
    stock_data['weekday'] = stock_data.Date.apply(lambda x : x.weekday())
    stock_data['weeknum'] = stock_data.Date.apply(lambda x : x.strftime('%V'))
    stock_data = stock_data.ffill()
    
    sample = pd.pivot_table(data = stock_data, values = 'Close', columns = 'weekday', index = 'weeknum').ffill()
    
    
    x = sample.iloc[0:-2].to_numpy()
    y = sample.iloc[1:-1].to_numpy()
    y_0 = y[:,0]
    y_1 = y[:,1]
    y_2 = y[:,2]
    y_3 = y[:,3]
    y_4 = y[:,4]
    
    print(f"type(y_0): {type(y_0)}")
    y_values = [y_0, y_1, y_2, y_3, y_4]
    x_public = sample.iloc[-2].to_numpy()
    
    return x, y_values, x_public

In [ ]:
def SDS_PCA_RF(x, y_values, x_public):
    for y_value in y_values:   
        pipe_rf = make_pipeline(StandardScaler(),PCA(n_components=3),
                            RandomForestRegressor(max_depth=100,random_state=1))

        pipe_rf.fit(x, y_value)
        prediction = pipe_rf.predict(np.expand_dims(x_public,0))

        predict_rf.append(prediction[0])
        print(f"{prediction}", end=' ')

In [ ]:
stock_list = pd.read_csv("./_data/stock_list.csv")
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x: str(x).zfill(6)) #zfill 숫자만큼 자릿수 맞춰 0 채우기
period = 365         # 1년간의 데이터를 사용
sample_submission = pd.read_csv("./_data/sample_submission.csv")
columns = list(sample_submission.columns[1:])
columns = ['Day'] + [str(x).zfill(6) for x in columns]
sample_submission.columns = columns
df = pd.DataFrame
  # 29일의 종가 예측
stock
for stock_code in stock_list['종목코드']:
    print(f"{stock_list[stock_list['종목코드']==stock_code]}")
    predict_rf = []
    print('-'*100)
    x, y_values, x_public = prep_data(stock_code, '20211101')
    SDS_PCA_RF(x, y_values, x_public)
    x, y_values, x_public = prep_data(stock_code, '20211129')
    SDS_PCA_RF(x, y_values, x_public)
    print()
    print('-'*100)
    sample_submission.loc[:,stock_code] = predict_rf

sample_submission.to_csv("./_data/submission.csv", index=False)

    